## 【高工局時間預測競賽】 國道一號10-99K回堵里程預測 - Feature Extraction
【Period】: 2024-07 ~ 2024-08 【Writer】: 李博業 **Bo-Ye,Li**

【Content】


- **天氣資訊(weather info)**
    - 針對特徵擷取分成天氣資訊以及車況資訊，天氣主要使用[timeanddate](https://www.timeanddate.com/)網站爬蟲資訊。
    1. 氣溫
    2. 氣壓
    3. 風速
    4. 濕度
- **交通即時資訊(traffic situation)**
    - 交通資訊選用[高工局](https://tisvcloud.freeway.gov.tw/)
    1. 事故前十分鐘車流量、大型車比例 
        - **使用各類車種通行量統計各類車種(M03A)計算**
    2. 事故後兩分鐘車流量 
        - **使用各類車種通行量統計各類車種(M03A)計算**
    3. 事故前十分鐘平均車速 
        - **各旅次路徑原始資料 (M06A)計算**

In [1]:
import numpy as np 
import pandas as pd
import glob
import os

#向被造訪的網站發出請求
import requests
import urllib

#靜態爬蟲
from bs4 import BeautifulSoup

#動態選單
from selenium import webdriver 
from selenium.webdriver.support.ui import Select 


from datetime import datetime, timedelta # 計算時間差、換成時間格式
import re  # 正則表達式(regular expression)
import time  # 時間

#解壓縮檔案
import tarfile 

#清理資料夾
import shutil


---

#### 【特徵擷取 1 - 天氣資訊】
- 至Time and Date網站中利用網路爬蟲，擷取所需氣候資訊

In [2]:
data = pd.read_excel(r"Data\original_data.xlsx")
data = data.head( 20 ) # 資料太多 先以前20筆為主
data

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,小客車,中大型客車,中小型貨車,大貨車及聯結車,其他車輛,追撞,自撞,起火,翻覆,其他事故
0,2023,1,1,9,39,國道 1 號,南,88.0,09:39,10:06,...,3,0,0,0,0,1,0,0,0,0
1,2023,1,1,10,34,國道 1 號,南,41.0,10:34,11:01,...,2,0,0,0,0,1,0,0,0,0
2,2023,1,1,11,15,國道 1 號,南,70.0,11:15,11:26,...,4,0,0,0,0,1,0,0,0,0
3,2023,1,1,15,46,國道 1 號,北,87.6,15:46,15:57,...,3,0,0,0,0,1,0,0,0,0
4,2023,1,1,17,17,國道 1 號,北,33.0,17:17,17:22,...,2,0,0,0,0,1,0,0,0,0
5,2023,1,1,18,18,國道 1 號,北,26.4,18:18,18:27,...,2,0,0,0,0,1,0,0,0,0
6,2023,1,1,18,31,國道 1 號,南,85.7,18:31,18:34,...,2,0,0,0,0,1,0,0,0,0
7,2023,1,1,22,37,國道 1 號,北,33.0,22:37,22:43,...,2,0,0,0,0,1,0,0,0,0
8,2023,1,2,6,17,國道 1 號,北,78.8,06:17,06:53,...,1,0,0,0,0,0,0,0,0,0
9,2023,1,2,9,49,國道 1 號,南,52.0,09:49,10:35,...,1,0,0,0,0,0,1,0,0,0


In [30]:
# 事件根據里程數對應之最近天氣測站

mile_dict = {
    "汐止" : "@1675393",
    "內湖" : "@6646680",
    "中山" : "@1677233",
    "三重" : "@1670141",
    "五股" : "@1665844",
    "林口" : "@1672882",
    "蘆竹" : "@1672580",
    "大園" : "@1667559",
    "中壢" : "@1677308",
    "平鎮" : "@6574590",
    "湖口" : "@1674410",
    "竹北" : "@1677111",
    "新竹東區" : "@11696361",
    "新竹縣寶山" : "@6600654" 
}

def miles_loc(miles):
    if miles >= 10 and miles <= 15: return "汐止"
    elif miles > 15 and miles <= 17: return "內湖"
    elif miles > 17 and miles <= 25: return "中山"
    elif miles > 25 and miles <= 27: return "三重"
    elif miles > 27 and miles <= 35: return "五股"
    elif miles > 35 and miles <= 41: return "林口"
    elif miles > 41 and miles <= 49: return "蘆竹"
    elif miles > 49 and miles <= 52: return "大園"
    elif miles > 52 and miles <= 62: return "中壢"
    elif miles > 62 and miles <= 71: return "平鎮"
    elif miles > 71 and miles <= 86: return "湖口"
    elif miles > 86 and miles <= 91: return "竹北"
    elif miles > 91 and miles <= 95: return "新竹東區"
    else: return "新竹縣寶山"



data["天氣查詢位置"] = data.iloc[:,7].apply(miles_loc)

In [31]:
cService = webdriver.ChromeService(r"ChromeDriver\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service = cService)  #打開Chrome


def weather_crawlertopage(year, month, day, loc_code:"str") -> "Data series":
    '''
    parameters: year 年分 , month 月份 , day 日期 , loc_mode "根據mile_dict搜索地區code"
    目的是先搜尋到所需的網頁
    '''

    url = f"https://www.timeanddate.com/weather/{loc_code}/historic?month={month}&year={year}"
    driver.get(url)

    choose_date = Select(driver.find_element("id", "wt-his-select"))
    choose_date.select_by_value(str(year)+"0"*(len(str(month))==1)+str(month)+"0"*(len(str(day))==1)+str(day))

    
def weather_dataframe_total(year, month, day, time_1, loc_code):
    '''
    爬取資料，回傳該事件天氣資訊
    '''
    weather_crawlertopage(year, month, day, loc_code)  #搜到所需網頁
    
    time.sleep(1.5)  #你要讓他睡一下覺 會有延遲 =_=
    
    
    #提取天氣資料表並做處理
    
    df_weather = pd.read_html(driver.page_source)[1]  

    df_weather.iloc[:,0] = df_weather.iloc[:,0].apply(lambda x: re.search("\d+:\d+", x).group()\
                                  if re.search("\d+:\d+", x) != None else None, 1)

    df_weather = df_weather.dropna(axis = 0, subset = [df_weather.columns[0]])
    df_weather.columns = ["時間", "天氣圖像", "溫度", "天氣描述", "風速", "風向", "濕度", "氣壓計", "能見度"]

    #尋找過去最近的時間
    sec_list = list(df_weather.iloc[:,0].apply\
                 (lambda x: abs((datetime.strptime(time_1, "%H:%M") - datetime.strptime(x, "%H:%M")).total_seconds())))  
    closese_time = np.argmin(sec_list)

    return df_weather.iloc[closese_time,:]

In [ ]:
#爬蟲並擷取氣候相關特徵的主程式
data["正常時間"] = data.apply(lambda x: "0"*(len(str(x["時"])) == 1)\
                          + str(x["時"]) + ":" + "0"*(len(str(x["分"])) == 1) + str(x["分"]) ,axis = 1)

data_weather_all = data.apply(lambda x : weather_dataframe_total(x["年"], x["月"] ,x["日"], x["正常時間"], mile_dict[x["天氣查詢位置"]]),\
                                                  axis=  1)
data_weather_all.drop(columns = ['天氣圖像','風向'])

In [34]:
print("--------擷取對應事件發生時之氣候資料-----------")
data_weather_all.head(10)

--------擷取對應事件發生時之氣候資料-----------


,時間,天氣圖像,溫度,天氣描述,風速,風向,濕度,氣壓計,能見度
0,09:30,NaN,21 °C,Broken clouds.,35 km/h,↑,78%,1024 mbar,NaN
1,10:30,NaN,21 °C,Partly sunny.,28 km/h,↑,78%,1024 mbar,NaN
2,11:00,NaN,21 °C,Partly sunny.,26 km/h,↑,78%,1024 mbar,NaN
3,16:00,NaN,21 °C,Broken clouds.,39 km/h,↑,69%,1021 mbar,NaN
4,17:30,NaN,19 °C,Passing clouds.,20 km/h,↑,78%,1022 mbar,NaN
5,18:30,NaN,19 °C,Passing clouds.,24 km/h,↑,83%,1023 mbar,NaN
6,18:30,NaN,20 °C,Passing clouds.,28 km/h,↑,78%,1022 mbar,NaN
7,22:30,NaN,19 °C,Partly cloudy.,19 km/h,↑,83%,1024 mbar,NaN
8,06:30,NaN,18 °C,Light rain. Partly cloudy.,13 km/h,↑,94%,1023 mbar,NaN
9,10:00,NaN,20 °C,Scattered showers. Partly sunny.,17 km/h,↑,88%,1025 mbar,NaN


---

#### 【特徵擷取2 - 交通即時資訊( 事故前十分鐘車流量、大型車比例 )】
- 使用M03A資料

In [3]:
#匯入測站資料(10K - 99K之間，頭尾有拉寬一些)
station_data = pd.read_excel(r"data/新-測站資料.xlsx")
station_data.loc[:,["ETagGantryID", "RoadName", "RoadDirection", "RoadSection/Start", "RoadSection/End", "LocationMile"]].\
            rename(columns = {"ETagGantryID":"測站代碼", "RoadName" : "國道", "RoadDirection":"方向",
                              "RoadSection/Start": "路段起點描述", "RoadSection/End":"路段迄點描述",
                              "LocationMile": "里程數"})

,測站代碼,國道,方向,路段起點描述,路段迄點描述,里程數
0,01F0061S,國道1號,S,大華系統,五堵,6.1
1,01F0061N,國道1號,N,五堵,大華系統,6.1
2,01F0099S,國道1號,S,五堵,汐止&amp;汐止系統,9.9
3,01F0099N,國道1號,N,汐止&amp;汐止系統,五堵,9.9
4,01F0147N,國道1號,N,東湖,汐止&amp;汐止系統,14.7
...,...,...,...,...,...,...
66,01F0980S,國道1號,S,新竹(科學園區),新竹系統,98.0
67,01F1045N,國道1號,N,頭份,新竹系統,104.5
68,01F1045S,國道1號,S,新竹系統,頭份,104.5
69,01F1123N,國道1號,N,頭屋,頭份,112.3


In [5]:
station_data.loc[:,["ETagGantryID", "RoadName", "RoadDirection", "RoadSection/Start", "RoadSection/End", "LocationMile"]]

,ETagGantryID,RoadName,RoadDirection,RoadSection/Start,RoadSection/End,LocationMile
0,01F0061S,國道1號,S,大華系統,五堵,6.1
1,01F0061N,國道1號,N,五堵,大華系統,6.1
2,01F0099S,國道1號,S,五堵,汐止&amp;汐止系統,9.9
3,01F0099N,國道1號,N,汐止&amp;汐止系統,五堵,9.9
4,01F0147N,國道1號,N,東湖,汐止&amp;汐止系統,14.7
...,...,...,...,...,...,...
66,01F0980S,國道1號,S,新竹(科學園區),新竹系統,98.0
67,01F1045N,國道1號,N,頭份,新竹系統,104.5
68,01F1045S,國道1號,S,新竹系統,頭份,104.5
69,01F1123N,國道1號,N,頭屋,頭份,112.3


In [6]:
def find_station_1(ran_pick_sing, index_num , station_data = station_data):
    '''
    Target: 尋找最近且可利用之測站
    
    Input:
       - ran_pick_sing: 是accident_data中每一row的資料。type : Series
       - station_data: 是總測站資料。type : DataFrame 
       - index_num: 要找事故發生地點的前第幾個測站。type: Int
    Output:
       - 回傳可用的最近測站名。type: str
    '''

    if ran_pick_sing['方向'] == '北':
        past_df = station_data.query(f"LocationMile >= {ran_pick_sing['里程']} and RoadDirection == 'N'")
        ind = list(np.argsort(past_df.loc[:,"LocationMile"]))
    else:
        past_df = station_data.query(f"LocationMile <= {ran_pick_sing['里程']} and RoadDirection == 'S'" )
        ind = list(np.argsort(past_df.loc[:,"LocationMile"]))[::-1]

    return past_df.iloc[ind[index_num],:]



In [7]:
def volumn_m03a(time_acci, station_1):
    '''
    爬蟲下載壓縮檔，解壓縮後為.tar.gz檔，查詢需要的csv files
    Input:
       - time_acci: 事故發生時間。type : datetime
       - station_1: 在沒壞掉測站中，離事故最近剛經過的測站。 type: str

    Output:
       - 輸出為兩個資料表，一為前五分鐘區間的資料，另一為前十分鐘區間的資料
    '''
    
    year = time_acci.year
    month = time_acci.month
    day = time_acci.day 

    #時間資料
    date_sliced_5 = time_acci - timedelta(minutes = (time_acci.minute % 5 + 5)) 
    date_sliced_10 = date_sliced_5 - timedelta(minutes = (5))
    

    #日期
    date = str(time_acci.date()).replace("-","")  #當下的時間
    date_5 = str(date_sliced_5.date()).replace("-","")   #前五分中的時間
    date_10 = str(date_sliced_10.date()).replace("-","") #前十分中的時間
    
    
    df = []
    members_list = [] #m03a前五 #m03a前十

    for date in [date_5, date_10]:

        file_name_m03a = r"download_data" + f"\\M03A_{date}.tar.gz"

        if os.path.isfile(file_name_m03a) == False:

            rr_m03a = requests.get(f"https://tisvcloud.freeway.gov.tw/history/TDCS/M03A/M03A_{date}.tar.gz")
            print(f"下載壓縮檔M03A_{date}")
            with open(file_name_m03a, "wb") as f:  #下載壓縮檔案(.tar.gz)
                f.write(rr_m03a.content)
        else:
            print(f"檔案已存在M03A_{date}")

        rr_1_m03a = tarfile.open(file_name_m03a, mode = "r:gz")   #打開壓縮檔

        # getmembers 來看看這個壓縮檔內有哪些成員 ! ><  

        #如果事件前五分鐘和前十分鐘是同一天就一起抓 然後break
        if date_5 == date_10:
            for z in [date_sliced_5, date_sliced_10]: 
                members_list.append([i for i in rr_1_m03a.getmembers() if (date + "_" + str(z.time()).replace(":","") +".csv") in str(i)][0])
        else:
            z = date_sliced_5 if date == date_5 else date_sliced_10
            members_list.append([i for i in rr_1_m03a.getmembers() if (date + "_" + str(z.time()).replace(":","") +".csv") in str(i)][0])
            
        
        start_index = 0*(date == date_5) + 1*(date != date_5)

        end_index = 2*(date_5 == date_10 or date == date_10) + 1*(date_5 != date_10 and date == date_5)
        
        
        for z in range(start_index,end_index):
            dd_ext = rr_1_m03a.extractfile(members_list[z])
            temp_dd = pd.read_csv(dd_ext, names=["時間", "測站", "方向", "車種", "數量"])
            df.append(temp_dd[(temp_dd["測站"] == station_1)])
           
        if date_5 == date_10: 
            break

    rr_1_m03a.close()  #關閉壓縮檔

    return df

In [40]:
accident_data = data 
accident_data.head(5)

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,中小型貨車,大貨車及聯結車,其他車輛,追撞,自撞,起火,翻覆,其他事故,天氣查詢位置,正常時間
0,2023,1,1,9,39,國道 1 號,南,88.0,09:39,10:06,...,0,0,0,1,0,0,0,0,竹北,09:39
1,2023,1,1,10,34,國道 1 號,南,41.0,10:34,11:01,...,0,0,0,1,0,0,0,0,林口,10:34
2,2023,1,1,11,15,國道 1 號,南,70.0,11:15,11:26,...,0,0,0,1,0,0,0,0,平鎮,11:15
3,2023,1,1,15,46,國道 1 號,北,87.6,15:46,15:57,...,0,0,0,1,0,0,0,0,竹北,15:46
4,2023,1,1,17,17,國道 1 號,北,33.0,17:17,17:22,...,0,0,0,1,0,0,0,0,五股,17:17


In [ ]:
#擷取該特徵的主程式

#要先放一個叫download_data的空資料夾存取爬蟲下來的壓縮檔

final = pd.DataFrame()   #紀錄事故前車流量

for tt_try in range(0, accident_data.shape[0]): #accident_data.shape[0]  # accident_data為所有事件資料
    
    ran_pick_sing = accident_data.loc[tt_try,:]

    #隨時清download_data的內容，才不會載太多東西
    if tt_try > 1 and accident_data.loc[tt_try-2,:]["日"] != accident_data.loc[tt_try,:]["日"]:
        shutil.rmtree(r"download_data")
        os.mkdir(r"download_data")

    #找station_1    
    time_acci = datetime.strptime(f"{ran_pick_sing['年']}/{ran_pick_sing['月']}/{ran_pick_sing['日']} {ran_pick_sing['正常時間']}", "%Y/%m/%d %H:%M") #ran_pick_sing["正常時間"] 

    station_1 = find_station_1(ran_pick_sing, 0)[0]

    df = volumn_m03a(time_acci, station_1)

    Found_lastone = "無"
    
    #檢查是否前十分鐘與前五分鐘的資料有缺
    
    if df[0].empty + df[1].empty > 0 :  #有缺
        Found_lastone = "有"
        station_1 = find_station_1(ran_pick_sing, 1)[0]  #找station_0
        df = volumn_m03a(time_acci, station_1)
            
    for a in range(0, 1+1): #reindex  
        df[a].index = [i for i in range(0,df[a].shape[0])]

    final_volumedf = df[1].copy()

    for i in range(0,final_volumedf.shape[0]):
        final_volumedf.loc[i,"時間"] = final_volumedf.loc[i,"時間"] + " & " + df[0].loc[i,"時間"]
        final_volumedf.loc[i,"數量"] = np.int64((df[0].query( f'車種 == {final_volumedf.loc[i,"車種"]}')["數量"] +\
                                        final_volumedf.loc[i,"數量"]))[0]

    car_type_dic = {    31:"事故前_小客車車流量",
                        32: "事故前_小貨車車流量",
                        41: "事故前_大客車車流量",
                        42: "事故前_大貨車流量",
                        5: "事故前_聯結車車流量" }

    temp_vv = final_volumedf.loc[:,["數量"]].T
    temp_vv.columns = [car_type_dic[final_volumedf.loc[i,"車種"]] for i in range(0, 4+1)]
    temp_vv.index = [0] 

    temp_vv["m03a_是否再往前找Station_1"] = Found_lastone

    temp_vv["事故前車流量_station1"] = station_1

    temp_vv["事故前_時間"] = final_volumedf.loc[0,"時間"]

    
    final = pd.concat([final, temp_vv], axis = 0)

In [45]:
print("特徵擷取2 - 交通即時資訊(事故前十分鐘車流量)")
final

特徵擷取2 - 交通即時資訊(事故前十分鐘車流量、大型車比例)


,事故前_小客車車流量,事故前_小貨車車流量,事故前_大客車車流量,事故前_大貨車流量,事故前_聯結車車流量,m03a_是否再往前找Station_1,事故前車流量_station1,事故前_時間
0,638,145,13,7,9,無,01F0880S,2023-01-01 09:25 & 2023-01-01 09:30
0,694,174,19,11,10,無,01F0376S,2023-01-01 10:20 & 2023-01-01 10:25
0,354,75,5,3,5,無,01F0699S,2023-01-01 11:05 & 2023-01-01 11:10
0,675,137,7,6,1,無,01F0880N,2023-01-01 15:35 & 2023-01-01 15:40
0,268,40,3,0,0,無,01H0333N,2023-01-01 17:05 & 2023-01-01 17:10
0,554,102,4,0,0,無,01H0271N,2023-01-01 18:05 & 2023-01-01 18:10
0,514,91,10,4,7,無,01F0750S,2023-01-01 18:20 & 2023-01-01 18:25
0,184,34,1,1,0,無,01H0333N,2023-01-01 22:25 & 2023-01-01 22:30
0,117,30,7,9,21,無,01F0880N,2023-01-02 06:05 & 2023-01-02 06:10
0,445,115,6,29,30,無,01F0511S,2023-01-02 09:35 & 2023-01-02 09:40


---

#### 【特徵擷取3 - 交通即時資訊( 事故後兩分鐘車流量)】
- 使用M03A資料、用權重的想法
    -  Ex: 32分事故，擷取30-35分的車流量並 * 2/5 
    -  35分事故，擷取35-40分的車流量並 * 2/5
    -  40分事故，擷取40-45分的車流量並 * 2/5
    -  29分事故，擷取25-30分的車流量並 * 1/5 + 擷取30-35分的車流量並 * 1/5
    -  34分事故，擷取30-35分的車流量並 * 1/5 + 擷取35-40分的車流量並 * 1/5

In [46]:
def afteracci_volumn(ran_pick_sing, station_1):
    

    time_acci = datetime.strptime(f"{ran_pick_sing['年']}/{ran_pick_sing['月']}/{ran_pick_sing['日']} {ran_pick_sing['正常時間']}", "%Y/%m/%d %H:%M") #ran_pick_sing["正常時間"] 


    date_sliced_1 = time_acci 
    date_sliced_1 = date_sliced_1 - timedelta(minutes = date_sliced_1.minute % 5)
    
    date_sliced_2 = time_acci + timedelta(minutes = 1)
    date_sliced_2 = date_sliced_2 - timedelta(minutes = date_sliced_2.minute % 5)
    
    date_1 = str(date_sliced_1.date()).replace("-","")
    date_2 = str(date_sliced_2.date()).replace("-","")
    
    
    df = []
    members_list = [] #m03a後一 #m03a 後二 

    end_index = 2*(date_1 == date_2) + 1*(date_1 != date_2)
    
    
    for date in [date_1, date_2]:

        file_name_m03a = r"download_data" + f"\\M03A_{date}.tar.gz"
        
        if os.path.isfile(file_name_m03a) == False:
            rr_m03a = requests.get(f"https://tisvcloud.freeway.gov.tw/history/TDCS/M03A/M03A_{date}.tar.gz")
            print("下載壓縮檔")
            with open(file_name_m03a, "wb") as f:  #下載壓縮檔案(.tar.gz)
                f.write(rr_m03a.content)
        else:
            print("檔案已存在")

        rr_1_m03a = tarfile.open(file_name_m03a, mode = "r:gz")   #打開壓縮檔
        
        # getmembers 來看看這個壓縮檔內有哪些成員 ! ><  

        if date_1 == date_2:
            for z in [date_sliced_1, date_sliced_2]: 
                    members_list.append([i for i in rr_1_m03a.getmembers() if (date + "_" + str(z.time()).replace(":","") +".csv") in str(i)][0])
        else:  #考慮不同天

            z = date_sliced_1 if date == date_1 else date_sliced_2
            members_list.append([i for i in rr_1_m03a.getmembers() if (date + "_" + str(z.time()).replace(":","") +".csv") in str(i)][0])
            
        
        
        start_index = 1*(date == date_2 and date_1 != date_2)

        end_index = 2*(date_1 == date_2 or date == date_2) + 1*(date_1 != date_2 and date == date_1)
        
        
        for z in range(start_index,end_index):
            dd_ext = rr_1_m03a.extractfile(members_list[z])
            temp_dd = pd.read_csv(dd_ext, names=["時間", "測站", "方向", "車種", "數量"])
            temp_dd = temp_dd[(temp_dd["測站"] == station_1)]
            
            df.append(temp_dd)

        
        if date_1 == date_2: 
            break
            
    rr_1_m03a.close()
    

    return df

In [ ]:
#擷取該特徵的主程式

final = pd.DataFrame() 

for index_df in range(0, accident_data.shape[0]):

    ran_pick_sing = accident_data.loc[index_df,:]

    if index_df > 1 and accident_data.loc[index_df-2,:]["日"] != accident_data.loc[index_df,:]["日"]:
        shutil.rmtree(r"download_data")
        os.mkdir(r"download_data")


    station_1 = find_station_1(ran_pick_sing, 0)[0]

    df = afteracci_volumn(ran_pick_sing, station_1)

    Found_lastone = "無"

    if df[0].empty + df[1].empty > 0 :
        Found_lastone = "有"
        station_1 = find_station_1(ran_pick_sing, 1)[0]
        df = afteracci_volumn(ran_pick_sing, station_1)
            
    for a in range(0, 1+1): #reindex  
        df[a].index = [i for i in range(0,df[a].shape[0])]
        
    df[0].數量 = df[0].數量.astype(np.float64)


    for i in range(0, 4+1):
        df[0].loc[i,"時間"] = df[0].loc[i,"時間"] + " & " + df[1].loc[i,"時間"]
        df[0].loc[i,"數量"] = np.float64(df[0].loc[i,"數量"]/5 +  df[1].query(f"車種 == {df[0].loc[i,'車種']}")\
        .loc[i,"數量"]/5)
                                        
    car_type_dic = {    31:"事故後_小客車流量",
                        32: "事故後_小貨車流量",
                        41: "事故後_大客車流量",
                        42: "事故後_大貨車流量",
                        5: "事故後_聯結車流量"}
                                        
    temp_v_after = df[0].loc[:,["數量"]].T
    temp_v_after.columns = [car_type_dic[df[0].loc[i,"車種"]] for i in range(0,4+1)]
    temp_v_after.index = [0] 

    final_data = pd.concat([pd.DataFrame(df[0].loc[0,["時間", "測站"]]).T, temp_v_after], axis = 1)
    final = pd.concat([final, final_data], axis = 0)

In [50]:
print("特徵擷取3 - 交通即時資訊(事故後兩分鐘車流量)")
final

特徵擷取3 - 交通即時資訊(事故後兩分鐘車流量)


,時間,測站,事故後_小客車流量,事故後_小貨車流量,事故後_大客車流量,事故後_大貨車流量,事故後_聯結車流量
0,2023-01-01 09:35 & 2023-01-01 09:40,01F0880S,103.6,25.6,1.2,1.0,0.6
0,2023-01-01 10:30 & 2023-01-01 10:35,01F0376S,144.8,31.6,3.6,3.0,1.4
0,2023-01-01 11:15 & 2023-01-01 11:15,01F0699S,66.4,17.6,2.0,3.6,0.4
0,2023-01-01 15:45 & 2023-01-01 15:45,01F0880N,106.0,27.6,0.8,2.8,0.0
0,2023-01-01 17:15 & 2023-01-01 17:15,01H0333N,46.0,10.8,0.0,0.0,0.0
0,2023-01-01 18:15 & 2023-01-01 18:15,01H0271N,112.4,16.0,0.0,0.0,0.0
0,2023-01-01 18:30 & 2023-01-01 18:30,01F0750S,101.2,18.4,2.8,2.0,0.8
0,2023-01-01 22:35 & 2023-01-01 22:35,01H0333N,42.0,4.8,0.0,0.0,0.0
0,2023-01-02 06:15 & 2023-01-02 06:15,01F0880N,33.6,5.6,0.4,2.0,2.8
0,2023-01-02 09:45 & 2023-01-02 09:50,01F0511S,85.8,26.8,0.8,4.6,5.6


---

#### 【特徵擷取4 - 交通即時資訊( 事故前十分鐘平均時速)】
- 利用政府M06A資料

為了在M06A查詢測站里程數需先匯入所有國道一號測站資料

In [56]:
#總測站資料
station_data_all = pd.read_csv(r"Data\門架總資訊.csv")
station_data_all = station_data_all.query("RoadName in ['國道1號', '國道1號汐止五股高架道路', '國道1號五股楊梅高架道路']")
station_data_all.LocationMile =  station_data_all.apply(lambda x: np.int16(x["LocationMile"].split("K+")[0])  + np.int16(x["LocationMile"].split("K+")[1])/1000 ,axis = 1)
station_data_all.head()


,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,RoadSection/Start,RoadSection/End,LocationMile
3,01F3227N,0000100132300D,4,120.25000,23.014172,000010,國道1號,0,N,大灣,永康,322.7
8,01F3227S,0000100032300D,4,120.25000,23.014172,000010,國道1號,0,S,永康,大灣,322.7
13,01F0532S,0000100005310H,4,121.26620,25.012943,000010,國道1號,0,S,機場系統,中壢服務區,53.2
14,01F0980S,0000100009800J,4,120.99886,24.763586,000010,國道1號,0,S,新竹(科學園區),新竹系統,98.0
15,01F1906N,0000100119050B,4,120.56813,24.113571,000010,國道1號,0,N,彰化系統,王田,190.6


In [57]:
#將資料轉換成字典型式
stat_allinfo_dict = {
    list(station_data_all.ETagGantryID)[i]:list(station_data_all.LocationMile)[i]  for i in range(0, station_data_all.shape[0])
    }

stat_allinfo_dict

{'01F3227N': 322.7,
 '01F3227S': 322.7,
 '01F0532S': 53.2,
 '01F0980S': 98.0,
 '01F1906N': 190.6,
 '01F0750S': 75.0,
 '01F0750N': 75.0,
 '01F0532N': 53.2,
 '01H0208N': 20.8,
 '01H0206S': 20.6,
 '01F3686S': 368.6,
 '01F1906S': 190.6,
 '01F3185N': 318.5,
 '01F0664S': 66.4,
 '01F2603S': 260.3,
 '01F3185S': 318.5,
 '01F2930S': 293.0,
 '01F2603N': 260.3,
 '01F2930N': 293.0,
 '01F0664N': 66.4,
 '01F1389N': 138.9,
 '01H0447N': 44.7,
 '01H0447S': 44.7,
 '01F1123S': 112.3,
 '01F1389S': 138.9,
 '01F1123N': 112.3,
 '01F0578S': 57.8,
 '01F0699S': 69.9,
 '01F0699N': 69.9,
 '01H0579S': 57.9,
 '01F1572N': 157.2,
 '01F3676N': 367.6,
 '01H0579N': 57.9,
 '01F1572S': 157.2,
 '01F3676S': 367.6,
 '01F3640S': 364.0,
 '01H0163S': 16.3,
 '01F3640N': 364.0,
 '01F2249N': 224.9,
 '01F2714S': 271.4,
 '01F0880N': 88.0,
 '01F0880S': 88.0,
 '01F2714N': 271.4,
 '01F2322S': 232.2,
 '01F2249S': 224.9,
 '01F2322N': 232.2,
 '01F3736S': 373.6,
 '01F3126N': 312.6,
 '01F2472S': 247.2,
 '01F3736N': 373.6,
 '01H0271N': 27.1,


In [54]:
#事件資料
acci_data = data
acci_data.head()

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,中小型貨車,大貨車及聯結車,其他車輛,追撞,自撞,起火,翻覆,其他事故,天氣查詢位置,正常時間
0,2023,1,1,9,39,國道 1 號,南,88.0,09:39,10:06,...,0,0,0,1,0,0,0,0,竹北,09:39
1,2023,1,1,10,34,國道 1 號,南,41.0,10:34,11:01,...,0,0,0,1,0,0,0,0,林口,10:34
2,2023,1,1,11,15,國道 1 號,南,70.0,11:15,11:26,...,0,0,0,1,0,0,0,0,平鎮,11:15
3,2023,1,1,15,46,國道 1 號,北,87.6,15:46,15:57,...,0,0,0,1,0,0,0,0,竹北,15:46
4,2023,1,1,17,17,國道 1 號,北,33.0,17:17,17:22,...,0,0,0,1,0,0,0,0,五股,17:17


M06_findspeed **計算平均時速之函式**

- input : 
    - time_acci : 事故發生時間。資料型態 : datetime
    - station_1 : 事故的station_1 (最近的ETAG測站)
    - stat_allinfo_dict : 查詢國道一號上所有測站的字典 
    
- output :
    - 該事件前十分鐘經過station_1的平均車速 -> 尋找的特徵交通即時資訊(事故前十分鐘平均時速)

In [61]:
def M06_findspeed(time_acci, station_1, stat_allinfo_dict = stat_allinfo_dict):
    
    
    first_hour = time_acci - timedelta(minutes = time_acci.minute)
    all_hour = [ first_hour - timedelta(hours = i ) for i in range(0, 4+1)]  #抓取前四個小時(包含此小時)的時間  
    record_one_speed = [] 

    for z in range(0, 4+1):
            date = re.search("\\d{8}", str(all_hour[z]).replace("-","")).group()
            tt = re.search("\\d{2}:\\d{2}:\\d{2}", str(all_hour[z]).replace("-","")).group().replace(":","")

            file_name_m06a = r"download_data" + f"\\M06A_{date}.tar.gz"

            if os.path.isfile(file_name_m06a) == False:   #如果資料夾沒有需要的檔案就要爬蟲下載

                rr_m06a = requests.get(f"https://tisvcloud.freeway.gov.tw/history/TDCS/M06A/M06A_{date}.tar.gz")
                print(f"下載壓縮檔 : M06A_{date}")
                with open(file_name_m06a, "wb") as f:  #下載壓縮檔案(.tar.gz)
                    f.write(rr_m06a.content)
                rr_m06a_1 = tarfile.open(file_name_m06a, mode = "r:gz")
            else: #如果資料夾有需要的檔案就open搜索需要的資料
                rr_m06a_1 = tarfile.open(file_name_m06a, mode = "r:gz")

            target_id = [i for i in rr_m06a_1.getmembers() if (date + "_" + tt +".csv") in str(i)][0]

            dd_ext = rr_m06a_1.extractfile(target_id)
            
            #temp_df是在第z小時的車子行駛資料
            
            temp_df = pd.read_csv(dd_ext, names = ["車種", "起點時間", "起點門架", "迄點時間", "迄點門架", "長度", "標記", "總資訊"])
            
            # 提取出有經過station1的車子 以及 車種為小客車
            temp_df = temp_df.query(f"總資訊.str.contains(@station_1) and 車種 == 31")  #
            
            #reindex 
            temp_df.index = [i for i in range(0, temp_df.shape[0])]
                
            for temp_index in range(0, temp_df.shape[0]):  # 對有經過station_1且為小客車的車子逐一檢查
                
                #拆解總資訊資料
                
                splited_list = temp_df.總資訊[temp_index].split(";")
                first_index = [i for i in range(0, len( splited_list )) if station_1 in splited_list[i]][0]
                time_station_1 = datetime.strptime(re.search("\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}:\\d{2}",splited_list[first_index]).group(),"%Y-%m-%d %H:%M:%S")
                
                #如果station_1不是list中的最後一個 且 在事故前十分鐘內經過station_1
                
                if first_index != (len(splited_list)-1) and (time_acci - time_station_1 ).seconds <= 600 :   
                    
                    #因為有可能有測站找不到里程的問題，所以才寫這個迴圈，找到可用的測站為止
                    
                    for second_find in range(first_index+1, len(splited_list)):  #如果一個station_2不行 試後面的

                        station_2 = splited_list[second_find].split("+")[1]
                    
                        if station_2 in stat_allinfo_dict.keys():  # 確保station_2是查的到里程的且能讓我計算車速的~

                            time_diff = (datetime.strptime(re.search("\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}:\\d{2}",splited_list[second_find]).group(),"%Y-%m-%d %H:%M:%S") -\
                                        time_station_1).seconds
                            
                            mile_diff = np.abs(stat_allinfo_dict[station_1] - stat_allinfo_dict[station_2])

                            speed = mile_diff/time_diff

                            record_one_speed.append(speed)
                            
                            break  #如果可以就break
                        
    rr_m06a_1.close() # 關閉打開的壓縮檔
    
    return record_one_speed  

In [ ]:
# 擷取該特徵的主程式

final = []

for index_df in range(0, 500+1):

    ran_pick_sing = acci_data.loc[index_df,:]

    if index_df > 1 and acci_data.loc[index_df-2,:]["日"] != acci_data.loc[index_df,:]["日"]:
    
        shutil.rmtree(r"download_data")
        os.mkdir(r"download_data")


    station_1 = find_station_1(ran_pick_sing, 0)[0]  # 尋找station_1

    #time_acci為事故發生時間
    time_acci = datetime.strptime(f"{ran_pick_sing['年']}/{ran_pick_sing['月']}/{ran_pick_sing['日']} {ran_pick_sing['正常時間']}", "%Y/%m/%d %H:%M") # ran_pick_sing["正常時間"] 
    
    #尋找平均車速
    ttmp = M06_findspeed(time_acci, station_1)
    
    if ttmp == []:
        station_1 = find_station_1(ran_pick_sing, 1)[0]
        ttmp = M06_findspeed(time_acci, station_1)

    final.append(round(np.mean(ttmp),4))

In [68]:
for i in range(0, 10):
    print(f"事件{i+1}前十分鐘平均時速: {final[i]} ")

事件1前十分鐘平均時速: 0.0244 
事件2前十分鐘平均時速: 0.0261 
事件3前十分鐘平均時速: 0.0256 
事件4前十分鐘平均時速: 0.0143 
事件5前十分鐘平均時速: 0.0228 
事件6前十分鐘平均時速: 0.0234 
事件7前十分鐘平均時速: 0.0256 
事件8前十分鐘平均時速: 0.0271 
事件9前十分鐘平均時速: 0.0268 
事件10前十分鐘平均時速: 0.0273 
